In [22]:
import pandas as pd
from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup

In [23]:
data = pd.read_excel("tweet1.xlsx")
#data

In [24]:
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"he’s":"he is","isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not", "might’ve": "might have","y’all":"you all","we're":"we are","you’re":"you "}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def clean_tweet(text):
    try:
        soup = BeautifulSoup(text,'lxml')
        souped = soup.get_text()
    except:
        souped = ""
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [25]:
from past.builtins import xrange
nums = [0,8791]
clean_tweet_texts = []
for i in xrange(nums[0],nums[1]):
    clean_tweet_texts.append(clean_tweet(data['tweet_text'][i]))

In [26]:
soup = BeautifulSoup("nan")
soup

<html><body><p>nan</p></body></html>

In [27]:
clean_tweet_texts

['also he is an unread malignant narcissist who forethought ends with the next day stock market numbers no way he would have done anything differently he literally has no ability to think past his immediate gratification think the caveman who sold square wheels to dumbers',
 'just swap the covid graphs with the stock market graphs boom problem solved',
 'so think might have waited to long to buy switch everywhere is sold out and people are charging an extra on amazon have no idea how the stock market works but bet it lot like this',
 'the stock market is beast like no other indifferent to human desires oblivious to common wisdom maddeningly contrary and seemingly bent on confounding the majority at every turn william neil stocks investing wjoquotes',
 'liquidity shall never freeze again ice stockmarket closes other markets closes where are the liquidity gonna go ripple bank crypto lending defi derivatives',
 'feeling optimistic that this will end short of months the stock market is reb

In [31]:
csv = 'clean_tweet_texts.csv'
new_df = pd.read_csv(csv,index_col=0)


In [30]:
join_df = pd.concat([data, new_df], axis = 1)
new_data = join_df [['got_criteria','text','date','hashtags','link']]
